# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [179]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk

In [180]:
from IPython import get_ipython
# Algumas configurações para o matplotlib.
# %matplotlib inline

# from IPython.core.pylabtools import figsize


# figsize(12, 8)

# sns.set()

In [181]:
countries = pd.read_csv("countries.csv")

In [182]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

In [183]:
# Sua análise começa aqui.

In [184]:
# Aqui vemos que realmente algumas variáveis numéricas estão sendo consideradas como 
# strings (objeto) por conta do uso da vírgula como seprador decimal.
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density          object
Coastline_ratio      object
Net_migration        object
Infant_mortality     object
GDP                 float64
Literacy             object
Phones_per_1000      object
Arable               object
Crops                object
Other                object
Climate              object
Birthrate            object
Deathrate            object
Agriculture          object
Industry             object
Service              object
dtype: object

In [185]:
# Irei então modificar o separador de vírgula para ponto:
countries.dtypes[countries.dtypes == object]

Country             object
Region              object
Pop_density         object
Coastline_ratio     object
Net_migration       object
Infant_mortality    object
Literacy            object
Phones_per_1000     object
Arable              object
Crops               object
Other               object
Climate             object
Birthrate           object
Deathrate           object
Agriculture         object
Industry            object
Service             object
dtype: object

In [186]:
# Essas são as colunas numéricas que estão codificadas como strings e que precisam ser alteradas 
# para numéricas (float):
numeric_features = ["Pop_density", "Coastline_ratio","Net_migration", "Infant_mortality",
                   "Literacy","Phones_per_1000", "Arable", "Crops", "Other","Climate","Birthrate", 
                   "Deathrate","Agriculture","Industry","Service"]

In [187]:
# Alteração das colunas:
countries[numeric_features] = countries[numeric_features].apply(lambda x: x.str.replace(',', '.').astype('float'))

In [188]:
# Aqui é possível verificar que as clunas foram alteradas e agora estão no formato adequado:
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density         float64
Coastline_ratio     float64
Net_migration       float64
Infant_mortality    float64
GDP                 float64
Literacy            float64
Phones_per_1000     float64
Arable              float64
Crops               float64
Other               float64
Climate             float64
Birthrate           float64
Deathrate           float64
Agriculture         float64
Industry            float64
Service             float64
dtype: object

In [189]:
# Agora corrigirei os espaços nas features "Country" e "Region":
columns = ["Country","Region"]
countries[columns] = countries[columns].apply(lambda x: x.str.strip())

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [192]:
def q1():
    return np.sort(list(countries.Region.unique()))

In [193]:
q1()

array(['ASIA (EX. NEAR EAST)', 'BALTICS', 'C.W. OF IND. STATES',
       'EASTERN EUROPE', 'LATIN AMER. & CARIB', 'NEAR EAST',
       'NORTHERN AFRICA', 'NORTHERN AMERICA', 'OCEANIA',
       'SUB-SAHARAN AFRICA', 'WESTERN EUROPE'], dtype='<U20')

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [194]:
from sklearn.preprocessing import KBinsDiscretizer

In [195]:
def q2():
    # Como o default da função KBinsDiscretizer é 'quantile', não preciso sinalisar esse parametro.
    data_discretized = KBinsDiscretizer(n_bins=10, encode='ordinal').fit_transform(countries[['Pop_density']])
    # Retornando os países se encontram acima do 90º percentil:
    return int((data_discretized > np.quantile(data_discretized, 0.9)).sum())

In [196]:
q2()

23

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [197]:
from sklearn.preprocessing import OneHotEncoder

In [198]:
def q3():
    encoder = OneHotEncoder(sparse=False, dtype=np.int)
    # Codificando a feature "Region":
    region_encoded = encoder.fit_transform(countries[["Region"]])
    # A feature climate possui missing values, preencherei esses valores com a média:
    countries["Climate"] = countries[['Climate']].fillna(countries['Climate'].mean())
    # Codificando a feature "Climate":
    climate_encoded = encoder.fit_transform(countries[["Climate"]])
    # Retornando a soma das features criadas:
    return region_encoded.shape[1] + climate_encoded.shape[1] 

In [199]:
q3()

18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [200]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [201]:
# !pip install -U scikit-learn

In [202]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [203]:
def q4():
    # Selecionando as variáveis do tipo int e float:
    variables = countries.select_dtypes(['int', 'float']).columns
    # Criando o pipeline:
    pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
    # Aplicando o pipeline nas variáveis:
    pipeline.fit(countries[variables])
    # Aplicando o pipeline no dado:
    pipe_data = pipeline.transform([test_country[2:]])
    # Retornando o valor da variável "Arable" após o pipeline. Lembrando que essa variável
    # é tem índice 9 em "variables":
    return float(pipe_data[0][9].round(3))

In [204]:
q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [205]:
def q5():
    # Cálculo do IQR (interquartile range):
    IQR = countries['Net_migration'].quantile(0.75) - countries['Net_migration'].quantile(0.25)
    # Cálculo do Q1:
    Q1 = countries['Net_migration'].quantile(0.25) - 1.5*IQR
    # Cálculo do Q2:
    Q2 = countries['Net_migration'].quantile(0.75) + 1.5*IQR
    # Definindo os ouliers abaixo e acima:
    outliers_abaixo = int((countries['Net_migration'] < Q1).sum())
    outliers_acima = int((countries['Net_migration'] > Q2).sum())
    return (outliers_abaixo, outliers_acima, False)

In [206]:
q5()

(24, 26, False)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [207]:
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import CountVectorizer

In [208]:
def q6():
    # Carregando as categorias:
    categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
    newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
    # Definindo o objeto vectorizer:
    vectorizer = CountVectorizer()
    # Aplicando o vectorizer ao data set newsgroups:
    vect_applied = vectorizer.fit_transform(newsgroup['data'])
    # Definindo a lista de palavras:
    words = vectorizer.get_feature_names() 
    # Contagem das palavras:
    count = vect_applied.toarray().sum(axis=0)
    # Dicionário com as palavras e a respectiva contagem:
    dic = dict(zip(words,count))
    # Retornando o número de vezes que a palavra "phone" aparece no corpus:
    return int(dic['phone'])

In [209]:
q6()

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [211]:
def q7():
    # Carregando as categorias:
    categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
    newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
    # Definindo o objeto vectorizer:
    vectorizer = TfidfVectorizer()
    # Aplicando o vectorizer ao data set newsgroups:
    vect_applied = vectorizer.fit_transform(newsgroup['data'])
    # Lista de palavras:
    words = vectorizer.get_feature_names();
    # Contagem das palavras:
    tf_idf = vect_applied.toarray().sum(axis=0)
    # Dicionário com as palavras e a respectiva TF-IDF(term frequency–inverse document frequency):
    dic = dict(zip(words,tf_idf))
    # Retornando o TF-IDF da palavra "phone":
    return float(dic['phone'].round(3))

In [212]:
q7()

8.888